# Eval Output Data

In [ ]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.font_manager
import seaborn as sns

# Ignore all warnings including PyPSA logged ones: most of these are known trivial
# ones about old versions, Notebook settings, etc. but note you should probably
# comment this out at points to check for useful warnings!
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger("pypsa").setLevel(level=logging.CRITICAL)  # ignore WARNING and below

# Bruno's starting ideas

In [ ]:
# datadir = '/home/users/train061/group_folder/data/'
datadir = '/gws/pw/j05/cop26_hackathons/oxford/Group_folders/group_5/data/'
model = 'ICHEC'
period = '1970-1976'

In [ ]:
network_d = datadir+'schlott_material/{}/{}'.format(model, period)

network = pypsa.Network(network_d)

## Features
Recommendation: Try to investigate measures which can be expressed as a function of the climate period (and potentially the node/carriers). Then, time series can be plotted and compared, similar periods can be identified, measures can be plotted against each other (scatter plots) to identify dependencies.

### Met Input Data

In [ ]:
### average capacity factors for resources wind (onwind + offwind), solar PV (solar) and runoff river (ror)

n_timesteps = len(network.snapshots)  # one value, don't plot

n_generators_per_carrier = pd.Series({carrier: len(network.generators.index[network.generators.carrier==carrier])
                                     for carrier in ['onwind', 'offwind', 'solar', 'ror']})

avg_capacity_factor = network.generators_t.p_max_pu.mean() # per generator

avg_capacity_factor_per_carrier = (network.generators_t.p_max_pu.groupby(network.generators.carrier, axis=1).sum().sum()/
                                   (n_timesteps*n_generators_per_carrier)) # per carrier

avg_capacity_factor_per_carrier_and_node = network.generators_t.p_max_pu.groupby([network.generators.bus, network.generators.carrier], 
                                                                                 axis=1).sum().sum()/n_timesteps # per carrier and bus

# Collect all values with their variable name as a key in a dict
###import sys
###sys.version_info  # should be in Python 3.8 so can use the below var name getting trick
av_cap_factors = {
    f'{n_generators_per_carrier=}'.split('=')[0]: n_generators_per_carrier,
    f'{avg_capacity_factor=}'.split('=')[0]: avg_capacity_factor,
    f'{avg_capacity_factor_per_carrier=}'.split('=')[0]: avg_capacity_factor_per_carrier,
    f'{avg_capacity_factor_per_carrier_and_node=}'.split('=')[0]: avg_capacity_factor_per_carrier_and_node,
}
print(av_cap_factors)

In [ ]:
### low-generation events (minimum power availability from VRES over a period of 2-4 days, see Kies_etal)
vregens = network.generators.index[network.generators.carrier.isin(['onwind','offwind','solar'])]
available_p = network.generators_t.p_max_pu[vregens].sum(axis=1)

minimum_p_available_pu = pd.Series({n_days: (available_p.rolling('{}D'.format(n_days)).sum()/(len(vregens)*n_days*8)).min()
                                    for n_days in range(2,5)})

# Collect all values with their variable name as a key in a dict
low_gen_events = {
    f'{vregens=}'.split('=')[0]: vregens,
    f'{available_p=}'.split('=')[0]: available_p,
    f'{minimum_p_available_pu=}'.split('=')[0]: minimum_p_available_pu,
}
print(av_cap_factors)

In [ ]:
### complementarity as described in .pptx in group folder
### for further complementarity indices see Jurasz_etal

complementarity_seasonal = pd.read_csv(datadir+'complementarity/{}/{}/seasonal.csv'.format(model, period), header=0, index_col=0) # only NaNs => Bruno

In [ ]:
### ramp rates (see Kies_etal)
### correlation lengths (see Schlott_etal)

### Simulation Outputs

In [ ]:
### capacity/generation shares
cap_share = network.generators.p_nom_opt.groupby(network.generators.carrier).sum() # per carrier
gen_share = network.generators_t.p.groupby(network.generators.carrier, axis=1).sum().sum() # per carrier
gen_share_per_node = network.generators_t.p.groupby([network.generators.bus, network.generators.carrier], axis=1).sum().sum() # per carrier and bus


# Collect all values with their variable name as a key in a dict
shares = {
    f'{cap_share=}'.split('=')[0]: cap_share,
    f'{gen_share=}'.split('=')[0]: gen_share,
    f'{gen_share_per_node=}'.split('=')[0]: gen_share_per_node,
}
print(shares)

In [ ]:
### levelized cost of electricity
investments = network.generators.p_nom_opt * network.generators.capital_cost
operation = network.generators_t.p.sum() * network.generators.marginal_cost

generation_per_carrier = network.generators_t.p.groupby(network.generators.carrier,axis=1).sum()
operation_per_carrier = operation.groupby(network.generators.carrier).sum()
investments_per_carrier = investments.groupby(network.generators.carrier).sum()

generation_per_carrier_and_node = network.generators_t.p.groupby([network.generators.bus, network.generators.carrier],axis=1).sum()
operation_per_carrier_and_node = operation.groupby([network.generators.bus, network.generators.carrier]).sum()
investments_per_carrier_and_node = investments.groupby([network.generators.bus, network.generators.carrier]).sum()

lcoe = (investments.sum() + operation.sum()) / network.loads_t.p_set.sum().sum() # for whole system  # one value, don't plot
lcoe_per_carrier = (investments_per_carrier.sum() + operation_per_carrier.sum()) / generation_per_carrier.sum() # per carrier
lcoe_per_carrier_and_node = ((investments_per_carrier_and_node.sum() + operation_per_carrier_and_node.sum()) / 
                    generation_per_carrier_and_node.sum()) # per carrier and bus


# Collect all values with their variable name as a key in a dict
costs = {
    f'{investments=}'.split('=')[0]: investments,
    f'{operation=}'.split('=')[0]: operation,
    f'{generation_per_carrier=}'.split('=')[0]: generation_per_carrier,
    f'{operation_per_carrier=}'.split('=')[0]: operation_per_carrier,
    f'{investments_per_carrier=}'.split('=')[0]: investments_per_carrier,
    f'{generation_per_carrier_and_node=}'.split('=')[0]: generation_per_carrier_and_node,
    f'{operation_per_carrier_and_node=}'.split('=')[0]: operation_per_carrier_and_node,
    f'{investments_per_carrier_and_node=}'.split('=')[0]: investments_per_carrier_and_node,
    f'{lcoe_per_carrier=}'.split('=')[0]: lcoe_per_carrier,
    f'{lcoe_per_carrier_and_node=}'.split('=')[0]: lcoe_per_carrier_and_node,
}
print(costs)

## Plotting of outputs for inspection of patterns etc.

**Please note** that these plots are generated via a minimal code loop from the variables above, to be created quickly for inspection and may not (in fact, probably are not) formatted nicely or in the right plot form (scatter, bar, etc.) for the given data (no comining of related data overlaid on one plot, etc.). The intention is that interesting data can be picked out and/or compared at a glance, and plotted elsewhere in a nicer and more appropriate fashion if relevant.

### Plots for the met input data

#### Average capacity factors:

In [ ]:
# Plot separate plots for each variable
for name, vals in av_cap_factors.items():
    fig, ax = plt.subplots(figsize=(18,10))

    # Do the actual plotting:
    if isinstance(vals, pd.core.series.Series):
        vals.plot()
    else:
        ax.plot(vals)

    ax.set(xlabel=name, ylabel='???', title=f"Plot of {name}")
    ax.grid()

    plt.show()
    fig.tight_layout()

#### Low generation events:

In [ ]:
# Plot separate plots for each variable
for name, vals in low_gen_events.items():
    fig, ax = plt.subplots(figsize=(18,10))

    # Do the actual plotting:
    if isinstance(vals, pd.core.series.Series):
        vals.plot()
    else:
        ax.plot(vals)

    ax.set(xlabel=name, ylabel='???', title=f"Plot of {name}")
    ax.grid()

    plt.show()
    fig.tight_layout()

#### Complementarity:

In [ ]:
# Plot separate plots for each variable
for name, vals in complementarity_seasonal.items():
    fig, ax = plt.subplots(figsize=(18,10))

    # Do the actual plotting:
    if isinstance(vals, pd.core.series.Series):
        vals.plot()
    else:
        ax.plot(vals)

    ax.set(xlabel=name, ylabel='???', title=f"Plot of {name}")
    ax.grid()

    plt.show()
    fig.tight_layout()

### Plots for the simulation outputs

#### Shares plots:

In [ ]:
# Plot separate plots for each variable
for name, vals in shares.items():
    fig, ax = plt.subplots(figsize=(18,10))

    # Do the actual plotting:
    if isinstance(vals, pd.core.series.Series):
        vals.plot()
    else:
        ax.plot(vals)

    ax.set(xlabel=name, ylabel='???', title=f"Plot of {name}")
    ax.grid()

    plt.show()
    fig.tight_layout()

#### Costs plots:

In [ ]:
# Plot separate plots for each variable
for name, vals in costs.items():
    fig, ax = plt.subplots(figsize=(18,10))

    # Do the actual plotting:
    if isinstance(vals, pd.core.series.Series):
        vals.plot()
    else:
        ax.plot(vals)

    ax.set(xlabel=name, ylabel='???', title=f"Plot of {name}")
    ax.grid()

    plt.show()
    fig.tight_layout()